# MSE
做regression時使用

In [10]:
import tensorflow as tf
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.025, 0.95, 0.025], [0.1, 0.8, 0.1]]

print(
f"MSE result: {tf.keras.losses.MSE(y_true, y_pred).numpy()}\
\nMAPE result: {tf.keras.losses.MAPE(y_true, y_pred).numpy()}\
\nMAE result: {tf.keras.losses.MAE(y_true, y_pred).numpy()}\
"
)

MSE result: [0.00125    0.48666668]
MAPE result: [1.6666669e+07 3.0000003e+08]
MAE result: [0.03333334 0.59999996]


# Cross-entropy
做分類問題時使用

In [2]:
import tensorflow as tf

y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.025, 0.95, 0.025], [0.1, 0.8, 0.1]]
# tf.keras.losses.categorical_crossentropy(y_pred = y_pred , y_true = y_true).numpy()
loss = tf.keras.losses.categorical_crossentropy(y_pred = y_pred , y_true = y_true)
assert loss.shape == (2,)
loss.numpy()

array([0.05129325, 2.3025851 ], dtype=float32)

In [14]:
# 也可以計算KLD
tf.keras.losses.KLD(y_pred = y_pred , y_true = y_true).numpy()

array([0.05129084, 2.3025823 ], dtype=float32)

In [50]:
# sparse 代表不做one-hot encoding
y_true = [1, 2]
y_pred = [[0.025, 0.95, 0.025], [0.1, 0.8, 0.1]]

tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred).numpy()

array([0.05129333, 2.3025851 ], dtype=float32)

In [54]:
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.025, 0.95, 0.025], [0.1, 0.8, 0.1]]

def cross_entropy(y_true, y_pred):
    x = -tf.math.reduce_sum(y_true * tf.math.log(y_pred), axis=1)
    return x.numpy()

print(cross_entropy(y_true, y_pred))


[0.05129331 2.3025851 ]


# DNN

In [5]:
import numpy as np
tf.compat.v1.reset_default_graph()
import tensorflow as tf
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()

        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):

        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]
    
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

# num_epochs = 5
batch_size = 5
# learning_rate = 0.001

model = MLP()
data_loader = MNISTLoader()

# num_batches = int(data_loader.num_train_data // batch_size * num_epochs)

X, y = data_loader.get_batch(batch_size)
print(y)
with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
    print(loss)

[6 8 4 1 5]
tf.Tensor([1.7521616 2.4806855 2.3468375 2.9898252 2.4064834], shape=(5,), dtype=float32)
